In [31]:

TRAIN_DATA_PATH = "data/train.txt"

In [33]:
# Sanity check, to ensure data is correct
def line_can_be_output(line):
    """
    Checks if line (list) is an output
    """
    for char in line:
        if char != '0' and char != '1': return False
        
    return True

train_file = open(TRAIN_DATA_PATH, "r")
file_lines = train_file.readlines()

for i in range(len(file_lines)):
    line = file_lines[i].replace("\n", "").split(" ")[:-1]
    if i % 32 > 16 and not line_can_be_output(line): # If it is a piece of output data
        print("Discrepancy in line " + str(i) + "!")
        
print(len(file_lines)/32)
train_file.close()

0


In [30]:
import numpy as np

"""
Returns data as an entire numpy array
"""
def matricize_data(file_path, num_data):
    data_file = open(file_path, "r")
    input_matrix = []
    label_matrix = []
    # Get data, 16 lines per
    for i in range(num_data):
        input_matrix.append([[int(number) for number in data_file.readline().replace("\n").split(" ")[:-1]] 
                            for i in range(16)])
        label_matrix.append([[int(number) for number in data_file.readline().replace("\n").split(" ")[:-1]] 
                        for i in range(16)])
    
    data_file.close()
    return (np.array(input_matrix), np.array(label_matrix))
    
"""
Generator that yields a piece of data
"""
def gen_matricize_data(file_path, num_data):
    data_file = open(file_path, "r")
    for i in range(num_data):
        input_matrix = [[int(number) for number in data_file.readline().replace("\n").split(" ")[:-1]] 
                            for i in range(16)]
        label_matrix = [[int(number) for number in data_file.readline().replace("\n").split(" ")[:-1]] 
                        for i in range(16)]
        yield (input_matrix, label_matrix)
    
    data_file.close()
    return